## Question 10 - (a)
### Theorem 8.8 give us a polynomial-time algorithm for finding a market equilibrium, which by Theorem 8.7 maximizes social value

## Question 10 - (b) Please Question 11 for VCG implementation

## Question 11 - (a) 
### Construct a graph of 20 buyers and 20 items. Just assign each player a random value per good (from 1 to 50). The valuation of buyers i to item(buldle) j is random_value * j 

In [48]:
import random

def create_graph(n):
    items = [i + 1 for i in range(n)]
    graph_valuation = []
    for player in range(1, n + 1):
        valuation_to_good = random.randint(1, 50)
        valuation_to_bundles = []

        for item in range(1, n + 1):
            valuation_to_bundles.append(valuation_to_good * item)
        graph_valuation.append(valuation_to_bundles)

        #print valuation_to_good
        #print valuation[player - 1]

    return graph_valuation

n = 20
graph_valuation = create_graph(n)
print 'graph_valuation: ', graph_valuation

graph_valuation:  [[35, 70, 105, 140, 175, 210, 245, 280, 315, 350, 385, 420, 455, 490, 525, 560, 595, 630, 665, 700], [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119, 126, 133, 140], [27, 54, 81, 108, 135, 162, 189, 216, 243, 270, 297, 324, 351, 378, 405, 432, 459, 486, 513, 540], [21, 42, 63, 84, 105, 126, 147, 168, 189, 210, 231, 252, 273, 294, 315, 336, 357, 378, 399, 420], [38, 76, 114, 152, 190, 228, 266, 304, 342, 380, 418, 456, 494, 532, 570, 608, 646, 684, 722, 760], [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100], [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000], [34, 68, 102, 136, 170, 204, 238, 272, 306, 340, 374, 408, 442, 476, 510, 544, 578, 612, 646, 680], [38, 76, 114, 152, 190, 228, 266, 304, 342, 380, 418, 456, 494, 532, 570, 608, 646, 684, 722, 760], [27, 54, 81, 108, 135, 162, 189, 216, 243, 270, 297, 324, 351, 378, 405, 432, 459, 486, 513, 540], [17, 34, 51, 6

## (b) Run  VCG on graph generated with 20 buyers and 20 items

In [49]:
import random
import copy
import numpy as np

def find_prefered_choice(valuations):
    
    prefered_choice = []
    for valuation in valuations:
        prefered_choiceId_player = [] 
        sorted_valuation = sorted(valuation)
        most_prefered = sorted_valuation[-1]
        for idx, choice in enumerate(valuation):
            if choice == most_prefered:
                prefered_choiceId_player.append(idx + 1)
            
        prefered_choice.append(prefered_choiceId_player)    
            
    return prefered_choice

def allocate_by_id(prefered_choices):
    
    #print '=============== allocate_by_id =============== \n'
    
    graph = Graph(prefered_choices)
    _, matching = graph.max_matching()
    # matching looks like [2, 0, 1]. 
    # It means item 0 is matched to buyer 2 and so on and so forth
    #print 'matching: ', matching
    
    # key: player, value: item id allocated
    allocation = {}
    for idx, value in enumerate(matching):
        if value == -1:
            continue
        allocation[value] = idx
    
    return allocation

def allocate_by_value(valuation, allocation_by_id):
    
    allocation_by_value = {}
    for playerId, itemId in allocation_by_id.iteritems():
        allocation_by_value[playerId] = valuation[playerId][itemId]
    
    return allocation_by_value

def create_graph(n):
    items = [i + 1 for i in range(n)]
    graph_valuation = []
    for player in range(1, n + 1):
        valuation_to_good = random.randint(1, 50)
        valuation_to_bundles = []

        for item in range(1, n + 1):
            valuation_to_bundles.append(valuation_to_good * item)
        graph_valuation.append(valuation_to_bundles)

        #print valuation_to_good
        #print valuation[player - 1]

    return graph_valuation

class Graph:
    
    def __init__(self, graph):
        #print graph
        self.graph = graph
        self.players = len(graph)
        self.items = len(graph[0])
        
    # A DFS based recursive function that returns true if a matching for u is possible
    def _matching(self, u, matching, visited):
        for v in range(self.items):
            # If player is interest in items v and v is None
            if self.graph[u][v] and visited[v] == False:
                visited[v] = True # Mark v as visited
                #If v is not assigned, or previous player had other choice
                if matching[v] == -1 or self._matching(matching[v], matching, visited):
                    matching[v] = u
                    return True
        return False
 
    # Returns maximum number of matching 
    def max_matching(self):
        #Show if the player was assigned or not.
        matching = [-1]*self.items
        #Count how many items that have been assigned.
        ans = 0
        for i in range(self.players):
            # Mark all items as not visited for next aplayersicant.
            visited = [False] * self.items
            # Find if a player can get a item
            if self._matching(i, matching, visited):
                ans += 1
        return ans, matching

    '''
    Returns true if there is a path from source 's' to sink 't' in
    residual graph. Also fills parent[] to store the path
    '''
    def bfs(self, s, t, parent):
 
        # Mark all vertices as not visited
        visited = [False] * (self.players)
        #print visited
         
        # Create a queue for BFS
        queue=[]
         
        # Mark the source node as visited and enqueue it
        queue.append(s)
        visited[s] = True
         
        # Standard BFS Loop
        while queue:
            #Dequeue a vertex from queue and print it
            u = queue.pop(0)
         
            # Get all neighbors of the dequeued vertex u
            # If the neighbor has not been visited, then mark it
            # visited and enqueue it
            for idx, val in enumerate(self.graph[u]):
                if visited[idx] == False and val > 0 :
                    queue.append(idx)
                    visited[idx] = True
                    parent[idx] = u
 
        # If we reached sink in BFS starting from source, then return
        # true, else false
        return True if visited[t] else False
    
    # Returns the maximum flow from s to t in the given graph
    def fordFulkerson(self, source, sink):
 
        # This array is filled by BFS and to store path
        parent = [-1] * (self.players)
 
        max_flow = 0 # There is no flow initially
 
        # Augment the flow while there is path from source to sink
        while self.bfs(source, sink, parent):
 
            # Find minimum residual capacity of the edges along the
            # path filled by BFS. Or we can say find the maximum flow
            # through the path found.
            path_flow = float("Inf")
            s = sink
            while (s != source):
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]
 
            # Add path flow to overall flow
            max_flow += path_flow
 
            # update residual capacities of the edges and reverse edges
            # along the path
            v = sink
            while (v != source):
                u = parent[v]
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]
 
        #print 'graph after: ', self.graph
        return max_flow, self.graph   

def create_expanded_graph(graph):

    # Construct G' with (1) node s and t, 
    # (2) edges with capacity 1 from s to every node in X
    # (3) edges with capacity 1 from every node in Y to t
    # (4) all edges in original G are set with capacity infinity
    x = len(graph)
    y = len(graph[0])
    n = x + y + 2
    #print '(x, y, n): ', (x, y, n)
    expG = []
    for i in range(n):

        #print 'connection for i: ', i 
        connection = []
        if i == 0: # node s
            # NOT connect s to itself
            connection.append(0)
            # connect s to all nodes in X
            for node in range(x):
                connection.append(1)
            # All nodes in Y is not connected to s
            for node in range(y):
                connection.append(0)
            # s is not connected to t
            connection.append(0) 
        elif i == n - 1: # node t
            # t is not connected to s
            connection.append(0)
            # All nodes in X is not connected to t
            for node in range(x):
                connection.append(0)
            # connect t to all nodes in Y
            for node in range(y):
                connection.append(1)
            # NOT connect t to itself
            connection.append(0)
        elif i >= 1 and i <= x: # nodes in X
            # connect to s
            connection.append(1)
            # NOT connect i in X to all nodes in X
            for node in range(x):
                connection.append(0)
            # connect i in X to nodes in Y according to original graph and set capacity as inf
            for node in range(y):
                if graph[i - 1][node] > 0:
                    connection.append(float('inf'))
                else:
                    connection.append(0)
            # NOT connect i in X to node t
            connection.append(0)
        elif i >= x + 1 and i <= 1 + x + y:
            # not connect i in Y to node s
            connection.append(0)
            # connect i in Y to nodes in X according to original graph and set capacity as inf
            for node in range(x):
                connection.append(0)
            # NOT connect i in Y to all nodes in Y
            for node in range(y):
                connection.append(0)
            # connect i in Y to node t
            connection.append(1)

        #print connection
        expG.append(connection)

    return expG

def create_prefered_choice_graph(valuations):

    n = len(valuations) # count of players
    m = len(valuations[0]) # count of items
    prefered_choices = []

    for valuation_player in valuations:
        most_prefered_value = max(valuation_player)
        prefered_choice = []
        for j in range(m):
            if valuation_player[j] == most_prefered_value:
                prefered_choice.append(1)
            else:
                prefered_choice.append(0)

        prefered_choices.append(prefered_choice)

    return prefered_choices

def find_constricted_set(prefered_choice_graph):
        
    expG = create_expanded_graph(prefered_choice_graph)
    #print '\n expG before: ', expG

    expGraph = Graph(expG)
    _, residual_graph = expGraph.fordFulkerson(0, len(expG) - 1)
    #print '\n residual_graph: ', residual_graph

    constricted_set = []
    reachable_from_source = []
    residual = Graph(residual_graph)

    for node in range(len(residual_graph)):
        parent = [-1] * (len(residual_graph))
        if residual.bfs(0, node, parent):
            #print node
            reachable_from_source.append(node - 1)

    #print '\n reachable_from_source: ', reachable_from_source

    for node in reachable_from_source:
        if node in range(len(prefered_choice_graph)):
            constricted_set.append(node)

    return constricted_set
    
def find_market_equilibrium(market):

    price = [0 for _ in range(len(market[0]))]

    while True:

        tmp_market = copy.deepcopy(market)
        for buyer in tmp_market:
            for item in range(len(market[0])):
                buyer[item] = buyer[item] - price[item]

        '''
        print '\n ==============================================='
        print '\n price: ', price
        print '\n tmp_market: ', tmp_market
        '''
        
        prefered_choice_graph = create_prefered_choice_graph(tmp_market)
        #print '\n prefered_choice_graph: ', prefered_choice_graph
        
        constricted_set = find_constricted_set(prefered_choice_graph)
        
        if len(constricted_set) == 0: # there exists a perfect matching
            #print "Detect perfect matching when finding market equilibrium, break!!!\n"
            break

        item_to_increase_price = []
        for buyer in constricted_set:
            #print 'buyer in constricted set: ', buyer
            for idx, value in enumerate(prefered_choice_graph[buyer]):
                #print 'idx, value: ', idx, value
                if value > 0 and idx not in item_to_increase_price:
                    item_to_increase_price.append(idx)
        
        #print 'item_to_increase_price: ', item_to_increase_price
        for item in item_to_increase_price:
            price[item] += 1
            
        if min(price) > 0:
            price = [item_price - 1 for item_price in price]

        #print 'price: ', price

    return prefered_choice_graph#price, tmp_market,

def vcg(valuation):
    
    n = len(valuation)
    
    prefered_choices_equilibrium = find_market_equilibrium(valuation)
    #print 'prefered_choices: \n', prefered_choices_equilibrium
    #print '\n'

    allocation_by_id = allocate_by_id(prefered_choices_equilibrium)
    #print 'allocation_by_id: \n', allocation_by_id 
    #print '\n'

    allocation_by_value = allocate_by_value(valuation, allocation_by_id)
    #print 'allocation_by_value: \n', allocation_by_value  
    #print '\n'
    
    maxSV = 0
    for key, values in allocation_by_value.iteritems():
        maxSV += values
    #print 'maxSV: \n', maxSV
    #print '\n'

    price_dict = {}
    #temp_eve_val_mat = np.array(valuation_value)
    
    for i in range(n):
        
        #print '\n\n====================================== vcg for player: ', i
        temp_valuation = copy.deepcopy(valuation)
        del temp_valuation[i]
        #print 'temp_valuation: ', temp_valuation 
    
        prefered_choices_equilibrium_without_i = find_market_equilibrium(temp_valuation)
        #print 'prefered_choices_equilibrium_without_i: \n', prefered_choices_equilibrium_without_i
        #print '\n'
        
        allocation_by_id_without_i = allocate_by_id(prefered_choices_equilibrium_without_i)
        #print 'allocation_by_id: \n', allocation_by_id_without_i 
        #print '\n'
        
        allocation_by_value_without_i = allocate_by_value(temp_valuation, allocation_by_id_without_i)
        #print 'allocation_by_value_without_i: \n', allocation_by_value_without_i  
        #print '\n'
        
        maxSV_without_i = 0
        for key, values in allocation_by_value_without_i.iteritems():
            maxSV_without_i += values
        #print 'maxSV_without_i: ', maxSV_without_i

        price_dict["Player" + str(i+1)] = maxSV_without_i - (maxSV - allocation_by_value[i])
        #print price_dict["Player" + str(i + 1)]

    print price_dict



In [56]:
n = 20
graph_valuation = create_graph(n)
#print 'graph_valuation: ', graph_valuation
price = vcg(graph_valuation)

{'Player14': 315, 'Player15': 160, 'Player16': 355, 'Player17': 49, 'Player10': 18, 'Player11': 11, 'Player12': 110, 'Player13': 4, 'Player18': 399, 'Player19': 63, 'Player2': 248, 'Player3': 187, 'Player1': 135, 'Player6': 0, 'Player7': 86, 'Player4': 444, 'Player5': 215, 'Player8': 281, 'Player9': 25, 'Player20': 36}


## (c) For each buyer, permute the list of valuations to each items(bundles)

In [57]:
from random import shuffle

def permute_valuation(valuation):
    
    permuted_valuation = []
    for buyer_valuation in valuation:
        shuffle(buyer_valuation)
    
    return valuation
    
n = 20
graph_valuation = create_graph(n)
#print 'graph_valuation: ', graph_valuation
permuted_valuation = permute_valuation(graph_valuation)
#print 'permuted_valuation: ', permuted_valuation

vcg(permuted_valuation)


{'Player14': 0, 'Player15': 49, 'Player16': 70, 'Player17': 24, 'Player10': 9, 'Player11': 24, 'Player12': 2, 'Player13': 11, 'Player18': 0, 'Player19': 46, 'Player2': 20, 'Player3': 1, 'Player1': 0, 'Player6': 7, 'Player7': 14, 'Player4': 29, 'Player5': 4, 'Player8': 22, 'Player9': 0, 'Player20': 0}
